# Sample Queries: Customer Behavior Genie

Sample queries organized by analytical category to help Genie learn query patterns and translate natural language to SQL.

**Purpose:** Reference queries for Customer Behavior Genie Room configuration

## Categories

1. Customer Segmentation & Value
2. Purchase Patterns & RFM Analysis
3. Product & Category Affinity
4. Channel Behavior & Migration
5. Engagement & Funnel Analysis
6. Abandonment & Recovery
7. Personalization & Affinity Impact
8. Stockout Risk Based on Customer Demand
9. Error Handling Examples
10. Multi-Domain Detection Examples

**Catalog/Schema:** Set in the **Configuration** cell below (session variables `catalog_name`, `schema_name`). Run that cell first, then run the query cells.


In [ ]:
-- =============================================================================
-- Configuration: Set target catalog and schema for all queries
-- Change these defaults to run against a different catalog/schema
-- =============================================================================
DECLARE OR REPLACE VARIABLE catalog_name STRING DEFAULT 'juan_use1_catalog';
DECLARE OR REPLACE VARIABLE schema_name STRING DEFAULT 'retail';

USE CATALOG IDENTIFIER(catalog_name);
USE SCHEMA IDENTIFIER(schema_name);

SELECT current_catalog(), current_schema(),
       catalog_name || '.' || schema_name AS target;

## 1. Customer Segmentation & Value


In [0]:
-- What are the key customer segments?
SELECT DISTINCT segment, COUNT(*) as customer_count
FROM gold_customer_dim
WHERE is_current = TRUE
GROUP BY segment
ORDER BY customer_count DESC;


segment,customer_count
regular,17500
loyal,12500
new,10000
premium,7500
vip,2500


In [0]:
-- What is the average lifetime value by customer segment?
SELECT 
    segment,
    AVG(lifetime_value) as avg_cltv,
    COUNT(*) as customer_count
FROM gold_customer_dim
WHERE is_current = TRUE
GROUP BY segment
ORDER BY avg_cltv DESC;


segment,avg_cltv,customer_count
vip,14775.6472,2500
premium,4993.883006666668,7500
loyal,1900.0410431999996,12500
regular,699.9626674285721,17500
new,277.736824,10000


In [0]:
-- What are the key customer segments, and how do their lifetime values differ?
SELECT 
    segment,
    COUNT(*) as customer_count,
    AVG(lifetime_value) as avg_cltv,
    MIN(lifetime_value) as min_cltv,
    MAX(lifetime_value) as max_cltv,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY lifetime_value) as median_cltv
FROM gold_customer_dim
WHERE is_current = TRUE
GROUP BY segment
ORDER BY avg_cltv DESC;


segment,customer_count,avg_cltv,min_cltv,max_cltv,median_cltv
vip,2500,14775.6472,5005.57,24993.51,14811.585
premium,7500,4993.883006666668,2000.06,7997.43,4986.485000000001
loyal,12500,1900.0410431999996,800.06,2999.99,1905.115
regular,17500,699.962667428572,200.01,1199.97,700.825
new,10000,277.73682400000007,50.02,499.95,279.46000000000004


In [0]:
-- Which segments have the highest average order value or frequency?
SELECT 
    c.segment,
    COUNT(DISTINCT s.transaction_id) / COUNT(DISTINCT c.customer_key) as avg_transactions_per_customer,
    AVG(s.net_sales_amount) as avg_order_value
FROM gold_customer_dim c
JOIN gold_sales_fact s ON c.customer_key = s.customer_key
WHERE c.is_current = TRUE
    AND s.is_return = FALSE
GROUP BY c.segment
ORDER BY avg_order_value DESC;

segment,avg_transactions_per_customer,avg_order_value
vip,2.4434861552428506,290.583743619064
premium,2.2797728667894415,285.23039955910014
loyal,2.3847907358438953,283.035495460198
regular,2.0976084538375974,279.24160189046825
new,1.9515244291363694,274.4267910545945


## 2. Purchase Patterns & RFM Analysis


In [0]:
-- How frequently do customers make purchases?
SELECT 
    customer_key,
    COUNT(DISTINCT transaction_id) as purchase_frequency
FROM gold_sales_fact
WHERE is_return = FALSE
GROUP BY customer_key
ORDER BY purchase_frequency DESC;

customer_key,purchase_frequency
19530,10
14499,9
14484,9
8462,9
34082,9
10553,9
36271,9
28467,9
15698,9
12213,9


In [0]:
-- Which customers are at risk of churning, and which are the most loyal?
WITH customer_rfm AS (
    SELECT 
        c.customer_key,
        c.segment,
        DATEDIFF(CURRENT_DATE, MAX(d.calendar_date)) as recency_days,
        COUNT(DISTINCT s.transaction_id) as frequency,
        SUM(s.net_sales_amount) as monetary_value
    FROM gold_customer_dim c
    JOIN gold_sales_fact s ON c.customer_key = s.customer_key
    JOIN gold_date_dim d ON s.date_key = d.date_key
    WHERE c.is_current = TRUE
        AND s.is_return = FALSE
        AND d.calendar_date >= DATE_SUB(CURRENT_DATE, 365)
    GROUP BY c.customer_key, c.segment
)
SELECT 
    customer_key,
    segment,
    recency_days,
    frequency,
    monetary_value,
    CASE 
        WHEN recency_days > 90 AND frequency < 3 THEN 'At Risk'
        WHEN recency_days <= 30 AND frequency >= 5 AND monetary_value > 1000 THEN 'Champion'
        WHEN recency_days <= 60 AND frequency >= 3 THEN 'Loyal'
        ELSE 'Regular'
    END as loyalty_status
FROM customer_rfm
ORDER BY 
    CASE loyalty_status
        WHEN 'At Risk' THEN 1
        WHEN 'Champion' THEN 2
        WHEN 'Loyal' THEN 3
        ELSE 4
    END,
    recency_days DESC;

customer_key,segment,recency_days,frequency,monetary_value,loyalty_status
27191,loyal,94,1,262.73,At Risk
21775,loyal,94,1,900.16,At Risk
35096,loyal,94,1,567.1,At Risk
45136,regular,94,1,212.08,At Risk
23546,new,94,1,77.98,At Risk
4081,new,94,1,367.12,At Risk
44355,loyal,94,1,351.38,At Risk
10759,loyal,94,1,1347.91,At Risk
49184,loyal,94,1,171.88,At Risk
21690,new,94,1,558.7,At Risk


In [0]:
-- Show me customers with low recency, high frequency, and high monetary value
WITH customer_rfm AS (
    SELECT 
        c.customer_key,
        c.customer_id,
        c.segment,
        DATEDIFF(CURRENT_DATE, MAX(d.calendar_date)) as recency,
        COUNT(DISTINCT s.transaction_id) as frequency,
        SUM(s.net_sales_amount) as monetary
    FROM gold_customer_dim c
    JOIN gold_sales_fact s ON c.customer_key = s.customer_key
    JOIN gold_date_dim d ON s.date_key = d.date_key
    WHERE c.is_current = TRUE
        AND s.is_return = FALSE
    GROUP BY c.customer_key, c.customer_id, c.segment
)
SELECT 
    customer_id,
    segment,
    recency,
    frequency,
    monetary
FROM customer_rfm
WHERE recency <= 30  -- Low recency (recent purchase)
    AND frequency >= 5  -- High frequency
    AND monetary >= 1000  -- High monetary value
ORDER BY monetary DESC;

customer_id,segment,recency,frequency,monetary
CUST_00039636,vip,19,7,15611.050000000001
CUST_00030446,vip,19,8,13921.929999999998
CUST_00004803,vip,20,6,12351.919999999998
CUST_00020464,vip,9,7,11434.560000000001
CUST_00044541,vip,9,6,11390.130000000001
CUST_00049217,vip,25,6,11191.94
CUST_00020592,vip,24,6,10967.7
CUST_00024592,vip,26,7,10882.07
CUST_00019434,vip,5,7,10881.789999999999
CUST_00014080,vip,4,6,10740.71


## 3. Product & Category Affinity


In [0]:
-- What products are trending?
SELECT 
    p.product_name,
    p.category_level_2,
    COUNT(DISTINCT s.transaction_id) as purchase_count,
    SUM(s.quantity_sold) as total_quantity,
    SUM(s.net_sales_amount) as total_revenue,
    COUNT(DISTINCT s.customer_key) as unique_customers
FROM gold_sales_fact s
JOIN gold_product_dim p ON s.product_key = p.product_key
JOIN gold_date_dim d ON s.date_key = d.date_key
WHERE s.is_return = FALSE
    AND d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
    AND p.is_active = TRUE
GROUP BY p.product_name, p.category_level_2
ORDER BY purchase_count DESC
LIMIT 20;

product_name,category_level_2,purchase_count,total_quantity,total_revenue,unique_customers
Urban Style T-Shirts 120,tops,78,165,16499.78000000001,78
Luxe Label Mini 737,dresses,77,160,8703.319999999998,77
Eco Threads Blazers 938,outerwear,76,149,10761.430000000004,76
Modern Minimal Sandals 1943,shoes,75,153,22162.09999999999,75
Classic Comfort Leggings 534,bottoms,74,141,32784.25999999999,74
Urban Style Silk 1540,scarves,74,144,19473.449999999993,74
Luxe Label T-Shirts 115,tops,73,147,34028.48000000001,73
Eco Threads Jackets 828,outerwear,72,136,5065.650000000002,72
Urban Style Sneakers 1748,shoes,71,145,35576.96000000001,71
Street Wear Co Wool 1555,scarves,71,149,6497.649999999998,71


In [0]:
-- Which product categories are most often purchased together?
WITH transaction_categories AS (
    SELECT DISTINCT
        s.transaction_id,
        p.category_level_2 as category
    FROM gold_sales_fact s
    JOIN gold_product_dim p ON s.product_key = p.product_key
    WHERE s.is_return = FALSE
)
SELECT 
    t1.category as category1,
    t2.category as category2,
    COUNT(DISTINCT t1.transaction_id) as co_occurrence_count
FROM transaction_categories t1
JOIN transaction_categories t2 ON t1.transaction_id = t2.transaction_id
WHERE t1.category < t2.category  -- Avoid duplicates and self-pairs
GROUP BY t1.category, t2.category
ORDER BY co_occurrence_count DESC
LIMIT 20;

category1,category2,co_occurrence_count
bottoms,tops,7473
shoes,tops,7395
bottoms,shoes,7317
bottoms,dresses,7285
dresses,shoes,7262
dresses,tops,7228
bottoms,scarves,6247
scarves,shoes,6069
scarves,tops,6064
dresses,scarves,5958


In [0]:
-- Show me products with high customer affinity scores
SELECT 
    p.product_name,
    p.category_level_2,
    AVG(a.affinity_score) as avg_affinity_score,
    COUNT(DISTINCT a.customer_key) as customer_count,
    SUM(a.purchase_count) as total_purchases
FROM gold_customer_product_affinity_agg a
JOIN gold_product_dim p ON a.product_key = p.product_key
WHERE a.affinity_score > 0.05  -- High affinity threshold
    AND p.is_active = TRUE
GROUP BY p.product_name, p.category_level_2
ORDER BY avg_affinity_score DESC, total_purchases DESC
LIMIT 20;

product_name,category_level_2,avg_affinity_score,customer_count,total_purchases
Modern Minimal Leggings 530,bottoms,0.150000,1,3
Bold Basics Handbags 1052,bags,0.100000,3,6
Vintage Vibes Boots 1763,shoes,0.100000,2,4
Eco Threads Bracelets 1368,jewelry,0.100000,2,4
Vintage Vibes Formal 637,dresses,0.100000,2,4
Bold Basics Mini 740,dresses,0.100000,2,4
Luxe Label T-Shirts 123,tops,0.100000,2,4
Eco Threads Blouses 60,tops,0.100000,2,4
Classic Comfort Totes 1260,bags,0.100000,2,4
Luxe Label Handbags 1072,bags,0.100000,2,4


In [0]:
-- How does category affinity differ by customer segment?
SELECT 
    c.segment,
    p.category_level_2,
    AVG(a.affinity_score) as avg_affinity_score,
    COUNT(DISTINCT a.customer_key) as customer_count,
    SUM(a.purchase_count) as total_purchases
FROM gold_customer_product_affinity_agg a
JOIN gold_customer_dim c ON a.customer_key = c.customer_key
JOIN gold_product_dim p ON a.product_key = p.product_key
WHERE c.is_current = TRUE
    AND p.is_active = TRUE
GROUP BY c.segment, p.category_level_2
ORDER BY c.segment, avg_affinity_score DESC;

segment,category_level_2,avg_affinity_score,customer_count,total_purchases
loyal,dresses,0.050043,6277,10099
loyal,jewelry,0.050043,4887,6945
loyal,scarves,0.050037,5543,8316
loyal,outerwear,0.050035,5382,8005
loyal,tops,0.050026,6349,10515
loyal,shoes,0.050024,6330,10411
loyal,bottoms,0.050009,6406,10560
loyal,bags,0.050005,4712,6584
new,dresses,0.049920,2634,3234
new,tops,0.049917,2825,3474


## 4. Channel Behavior & Migration


In [0]:
-- Through which channels do customers prefer to shop?
SELECT 
    ch.channel_name,
    COUNT(DISTINCT s.customer_key) as unique_customers,
    COUNT(DISTINCT s.transaction_id) as transaction_count,
    SUM(s.net_sales_amount) as total_revenue,
    AVG(s.net_sales_amount) as avg_order_value
FROM gold_sales_fact s
JOIN gold_channel_dim ch ON s.channel_key = ch.channel_key
JOIN gold_customer_dim c ON s.customer_key = c.customer_key
WHERE s.is_return = FALSE
    AND c.is_current = TRUE
GROUP BY ch.channel_name
ORDER BY total_revenue DESC;

channel_name,unique_customers,transaction_count,total_revenue,avg_order_value
Website - Desktop,18727,32550,2.0988100729999445E7,282.4169859787858
Physical Store - POS,9692,16800,1.5772949039999694E7,284.7719549360817
iOS Mobile App,13730,22581,1.3169852429999672E7,281.6658987958952
Facebook Shop,5118,6598,3377069.8699999927,276.89979255493546
In-Store Kiosk,3239,3365,2152228.190000005,287.30852890134895
Android Mobile App,3255,3361,2092513.039999994,281.44089307330114
Website - Mobile,3152,3263,2037569.009999995,279.3103509252906
Amazon Marketplace,3248,3344,2034009.0500000007,278.63137671232886


In [0]:
-- How do customers migrate between acquisition and preferred channels over time?
SELECT 
    c.acquisition_channel,
    c.preferred_channel,
    COUNT(*) as customer_count,
    AVG(c.lifetime_value) as avg_cltv
FROM gold_customer_dim c
WHERE c.is_current = TRUE
    AND c.acquisition_channel IS NOT NULL
    AND c.preferred_channel IS NOT NULL
GROUP BY c.acquisition_channel, c.preferred_channel
ORDER BY customer_count DESC;

acquisition_channel,preferred_channel,customer_count,avg_cltv
web,web,7067,1764.224363945097
social,web,5406,2708.2844395116535
social,app,5142,1689.0475418125252
web,app,4935,1713.5760364741634
search,app,3755,602.0930199733689
search,web,3730,603.1971608579089
store,web,3708,4413.702003775622
store,store,2451,3911.804169726642
email,web,2183,1894.6860604672468
email,store,2111,1895.9158313595449


## 5. Engagement & Funnel Analysis


In [0]:
-- What are the conversion rates at each stage of the engagement funnel?
WITH funnel_metrics AS (
    SELECT 
        COUNT(DISTINCT CASE WHEN e.event_type = 'view' THEN e.session_id END) as views,
        COUNT(DISTINCT CASE WHEN e.event_type = 'add_to_cart' THEN e.session_id END) as add_to_cart,
        COUNT(DISTINCT CASE WHEN s.transaction_id IS NOT NULL THEN e.session_id END) as purchases
    FROM gold_customer_event_fact e
    LEFT JOIN gold_sales_fact s ON e.customer_key = s.customer_key 
        AND e.date_key = s.date_key
    JOIN gold_date_dim d ON e.date_key = d.date_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
)
SELECT 
    views,
    add_to_cart,
    purchases,
    (add_to_cart::DOUBLE / NULLIF(views, 0)) * 100 as cart_conversion_rate,
    (purchases::DOUBLE / NULLIF(add_to_cart, 0)) * 100 as purchase_conversion_rate,
    (purchases::DOUBLE / NULLIF(views, 0)) * 100 as overall_conversion_rate
FROM funnel_metrics;

views,add_to_cart,purchases,cart_conversion_rate,purchase_conversion_rate,overall_conversion_rate
0,1178,333,null,28.268251273344653,null


In [0]:
-- Where are customers dropping off in the purchase funnel?
WITH funnel_stages AS (
    SELECT 
        COUNT(DISTINCT CASE WHEN e.event_type = 'view' THEN e.session_id END) as stage1_views,
        COUNT(DISTINCT CASE WHEN e.event_type = 'add_to_cart' THEN e.session_id END) as stage2_cart,
        COUNT(DISTINCT CASE WHEN s.transaction_id IS NOT NULL THEN e.session_id END) as stage3_purchase
    FROM gold_customer_event_fact e
    LEFT JOIN gold_sales_fact s ON e.customer_key = s.customer_key 
        AND e.date_key = s.date_key
        AND s.is_return = FALSE
    JOIN gold_date_dim d ON e.date_key = d.date_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
)
SELECT 
    stage1_views,
    stage2_cart,
    stage3_purchase,
    (stage1_views - stage2_cart) as drop_off_views_to_cart,
    (stage2_cart - stage3_purchase) as drop_off_cart_to_purchase,
    ((stage1_views - stage2_cart)::DOUBLE / NULLIF(stage1_views, 0)) * 100 as drop_off_rate_views_to_cart,
    ((stage2_cart - stage3_purchase)::DOUBLE / NULLIF(stage2_cart, 0)) * 100 as drop_off_rate_cart_to_purchase
FROM funnel_stages;

stage1_views,stage2_cart,stage3_purchase,drop_off_views_to_cart,drop_off_cart_to_purchase,drop_off_rate_views_to_cart,drop_off_rate_cart_to_purchase
0,1178,302,-1178,876,null,74.36332767402376


## 6. Abandonment & Recovery


In [0]:
-- What is the rate of cart abandonment?
SELECT 
    COUNT(*) as total_carts,
    SUM(CASE WHEN is_recovered = FALSE THEN 1 ELSE 0 END) as abandoned_carts,
    (SUM(CASE WHEN is_recovered = FALSE THEN 1 ELSE 0 END)::DOUBLE / COUNT(*)) * 100 as abandonment_rate
FROM gold_cart_abandonment_fact;

total_carts,abandoned_carts,abandonment_rate
3509,3424,97.5776574522656


In [0]:
-- What is the rate of cart abandonment, and how effective are recovery campaigns?
SELECT 
    COUNT(*) as total_abandonments,
    SUM(CASE WHEN recovery_email_sent = TRUE THEN 1 ELSE 0 END) as recovery_emails_sent,
    SUM(CASE WHEN is_recovered = TRUE THEN 1 ELSE 0 END) as recovery_conversions,
    (SUM(CASE WHEN is_recovered = FALSE THEN 1 ELSE 0 END)::DOUBLE / COUNT(*)) * 100 as abandonment_rate,
    (SUM(CASE WHEN is_recovered = TRUE THEN 1 ELSE 0 END)::DOUBLE / 
     NULLIF(SUM(CASE WHEN recovery_email_sent = TRUE THEN 1 ELSE 0 END), 0)) * 100 as recovery_conversion_rate
FROM gold_cart_abandonment_fact;

total_abandonments,recovery_emails_sent,recovery_conversions,abandonment_rate,recovery_conversion_rate
3509,2850,85,97.5776574522656,2.982456140350877


In [0]:
-- Show me cart abandonment rates by customer segment
SELECT 
    c.segment,
    COUNT(*) as total_abandonments,
    SUM(CASE WHEN ca.is_recovered = FALSE THEN 1 ELSE 0 END) as abandoned_carts,
    (SUM(CASE WHEN ca.is_recovered = FALSE THEN 1 ELSE 0 END)::DOUBLE / COUNT(*)) * 100 as abandonment_rate,
    AVG(ca.cart_value) as avg_cart_value
FROM gold_cart_abandonment_fact ca
JOIN gold_customer_dim c ON ca.customer_key = c.customer_key
WHERE c.is_current = TRUE
GROUP BY c.segment
ORDER BY abandonment_rate DESC;

segment,total_abandonments,abandoned_carts,abandonment_rate,avg_cart_value
regular,1178,1161,98.55687606112055,278.4205517826826
premium,501,491,98.00399201596807,272.31766467065864
vip,189,184,97.35449735449735,281.71354497354497
new,705,684,97.02127659574468,275.31668085106384
loyal,936,904,96.58119658119658,274.53769230769234


In [0]:
-- What is the revenue recovery rate from abandoned carts?
SELECT 
    COUNT(*) as total_abandonments,
    SUM(cart_value) as total_abandoned_value,
    SUM(CASE WHEN is_recovered = TRUE THEN cart_value ELSE 0 END) as recovered_revenue,
    (SUM(CASE WHEN is_recovered = TRUE THEN cart_value ELSE 0 END)::DOUBLE / 
     NULLIF(SUM(cart_value), 0)) * 100 as revenue_recovery_rate
FROM gold_cart_abandonment_fact;

total_abandonments,total_abandoned_value,recovered_revenue,revenue_recovery_rate
3509,968719.96,25480.079999999994,2.630283368993449


## 7. Personalization & Affinity Impact


In [0]:
-- Which segments respond best to personalization?
SELECT 
    c.segment,
    AVG(a.affinity_score) as avg_affinity_score,
    COUNT(DISTINCT a.customer_key) as customers_with_affinity,
    SUM(a.purchase_count) as total_purchases,
    AVG(a.predicted_cltv_impact) as avg_predicted_cltv_impact
FROM gold_customer_product_affinity_agg a
JOIN gold_customer_dim c ON a.customer_key = c.customer_key
WHERE c.is_current = TRUE
    AND a.affinity_score > 0.05  -- High affinity threshold
GROUP BY c.segment
ORDER BY avg_affinity_score DESC;

segment,avg_affinity_score,customers_with_affinity,total_purchases,avg_predicted_cltv_impact
vip,0.100833,58,121,708.7988333333335
new,0.100000,24,48,617.4479166666666
premium,0.100000,80,160,591.88775
regular,0.099636,55,109,595.7074545454547
loyal,0.099587,121,240,752.0245454545452


In [0]:
-- What is the predicted impact of affinity-based targeting on CLTV?
SELECT 
    c.segment,
    COUNT(DISTINCT a.customer_key) as targeted_customers,
    SUM(a.predicted_cltv_impact) as total_predicted_cltv_impact,
    AVG(a.predicted_cltv_impact) as avg_predicted_cltv_impact,
    AVG(c.lifetime_value) as current_avg_cltv
FROM gold_customer_product_affinity_agg a
JOIN gold_customer_dim c ON a.customer_key = c.customer_key
WHERE c.is_current = TRUE
    AND a.affinity_score > 0.05  -- High affinity threshold
GROUP BY c.segment
ORDER BY total_predicted_cltv_impact DESC;

segment,targeted_customers,total_predicted_cltv_impact,avg_predicted_cltv_impact,current_avg_cltv
loyal,121,90994.96999999997,752.0245454545452,1873.303719008265
premium,80,47351.02,591.88775,5149.438625000001
vip,58,42527.93000000001,708.7988333333335,14231.28583333333
regular,55,32763.910000000007,595.7074545454547,707.7889090909092
new,24,14818.749999999998,617.4479166666666,256.31250000000006


## 8. Stockout Risk Based on Customer Demand


In [0]:
WITH demand_signals AS (
    SELECT 
        p.product_key,
        p.product_name,
        p.category_level_2,
        COUNT(DISTINCT s.transaction_id) as recent_purchase_count,
        SUM(s.quantity_sold) as recent_demand,
        first(
            (
                SELECT COUNT(*) 
                FROM gold_cart_abandonment_fact ca2
                JOIN gold_customer_dim c2 ON ca2.customer_key = c2.customer_key
                WHERE ca2.low_inventory_trigger = TRUE
                  AND c2.preferred_category = p.category_level_2
            )
        ) as low_inventory_abandonments
    FROM gold_sales_fact s
    JOIN gold_product_dim p ON s.product_key = p.product_key
    JOIN gold_date_dim d ON s.date_key = d.date_key
    WHERE s.is_return = FALSE
        AND p.is_active = TRUE
        AND d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
    GROUP BY p.product_key, p.product_name, p.category_level_2
)
SELECT 
    product_name,
    category_level_2,
    recent_purchase_count,
    recent_demand,
    low_inventory_abandonments,
    CASE 
        WHEN low_inventory_abandonments > 10 AND recent_demand > 100 THEN 'Stockout Risk'
        WHEN recent_demand < 10 THEN 'Overstock Risk'
        ELSE 'Normal'
    END as risk_level
FROM demand_signals
WHERE low_inventory_abandonments > 0 OR recent_demand < 10
ORDER BY low_inventory_abandonments DESC, recent_demand DESC;

product_name,category_level_2,recent_purchase_count,recent_demand,low_inventory_abandonments,risk_level
Urban Style T-Shirts 120,tops,78,165,501,Stockout Risk
Luxe Label T-Shirts 115,tops,73,147,501,Stockout Risk
Bold Basics T-Shirts 122,tops,69,143,501,Stockout Risk
Modern Minimal T-Shirts 117,tops,65,138,501,Stockout Risk
Urban Style Blouses 63,tops,62,134,501,Stockout Risk
Urban Style Blouses 79,tops,64,132,501,Stockout Risk
Street Wear Co Hoodies 225,tops,64,132,501,Stockout Risk
Classic Comfort Sweaters 195,tops,65,132,501,Stockout Risk
Vintage Vibes T-Shirts 114,tops,64,131,501,Stockout Risk
Vintage Vibes Shirts 23,tops,60,130,501,Stockout Risk


## 9. Error Handling Examples

These queries demonstrate error scenarios that Genie should handle gracefully.


In [0]:
-- Show me customer purchase data from schema X (permission error example)
-- This query would fail if user lacks permissions
-- Genie should detect and provide actionable guidance
-- SELECT * FROM unauthorized_schema.customer_data;

In [0]:
-- Show me customer data (ambiguous query example)
-- This query is too ambiguous - could mean segments, purchases, or engagement
-- Genie should ask for clarification or provide multiple interpretations
SELECT * FROM gold_customer_dim WHERE is_current = TRUE LIMIT 100;

customer_key,customer_id,email,first_name,last_name,segment,lifetime_value,acquisition_date,acquisition_channel,preferred_channel,preferred_category,size_profile_tops,size_profile_bottoms,geo_region,geo_city,nearest_store_id,loyalty_tier,email_subscribe_flag,sms_subscribe_flag,effective_date,expiration_date,is_current,source_system,etl_timestamp
1,CUST_00000001,danielle.johnson857@hotmail.com,Danielle,Johnson,premium,4021.36,2021-05-14,store,web,dresses,XS,38,West,Seattle,4,silver,true,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.098Z
2,CUST_00000002,joshua.walker266@icloud.com,Joshua,Walker,regular,256.48,2022-11-07,search,app,accessories,XXL,26,Midwest,Chicago,5,bronze,true,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.098Z
3,CUST_00000003,jill.rhodes619@yahoo.com,Jill,Rhodes,regular,615.61,2021-12-25,web,app,accessories,S,36,Southeast,Tampa,10,silver,true,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.098Z
4,CUST_00000004,patricia.miller530@outlook.com,Patricia,Miller,regular,792.72,2022-10-31,search,web,bottoms,XS,30,Midwest,Cleveland,10,silver,false,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.098Z
5,CUST_00000005,robert.johnson188@hotmail.com,Robert,Johnson,regular,1173.95,2022-09-25,social,web,tops,XXL,30,Midwest,Cleveland,10,bronze,true,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.099Z
6,CUST_00000006,jeffery.wagner522@icloud.com,Jeffery,Wagner,regular,771.92,2022-09-16,social,app,bottoms,XS,30,Southeast,Atlanta,2,bronze,true,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.099Z
7,CUST_00000007,anthony.gonzalez512@outlook.com,Anthony,Gonzalez,regular,682.02,2021-07-02,search,web,accessories,XL,28,Midwest,Chicago,9,silver,true,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.099Z
8,CUST_00000008,debra.gardner771@icloud.com,Debra,Gardner,new,88.23,2021-12-14,paid,social,bottoms,M,30,West,San Francisco,3,bronze,false,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.099Z
9,CUST_00000009,jeffrey.lawrence503@hotmail.com,Jeffrey,Lawrence,new,350.68,2022-08-22,paid,web,tops,XXL,30,Northeast,Philadelphia,3,bronze,false,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.099Z
10,CUST_00000010,lisa.smith875@hotmail.com,Lisa,Smith,new,247.47,2022-05-05,paid,web,bottoms,XXL,32,Midwest,Detroit,6,bronze,false,false,2023-01-01,9999-12-31,true,CRM_SALESFORCE,2025-12-25T16:31:18.099Z


## 10. Multi-Domain Detection Examples

These queries should be detected as multi-domain and redirected to the multi-domain agent.


In [0]:
-- Which products are frequently abandoned in carts and do we have inventory issues?
-- This query requires inventory data (multi-domain)
-- Genie should detect keywords like "inventory issues" and redirect
SELECT 
    abandonment_stage,
    COUNT(*) as abandonment_count,
    SUM(CASE WHEN low_inventory_trigger = TRUE THEN 1 ELSE 0 END) as low_inventory_abandonments
FROM gold_cart_abandonment_fact
GROUP BY abandonment_stage
ORDER BY abandonment_count DESC;
-- Note: For product-level abandonment and inventory data, use multi-domain agent

abandonment_stage,abandonment_count,low_inventory_abandonments
payment,1194,534
cart,1183,509
shipping,1132,485


In [0]:
-- Show me products with high cart abandonment and their current stock levels
-- This query requires inventory data (multi-domain)
-- Genie should detect "stock levels" and redirect
SELECT 
    abandonment_stage,
    COUNT(*) as abandonment_count,
    AVG(cart_value) as avg_cart_value,
    SUM(CASE WHEN low_inventory_trigger = TRUE THEN 1 ELSE 0 END) as low_inventory_abandonments
FROM gold_cart_abandonment_fact
GROUP BY abandonment_stage
ORDER BY abandonment_count DESC;
-- Note: Stock levels require inventory data - use multi-domain agent

abandonment_stage,abandonment_count,avg_cart_value,low_inventory_abandonments
payment,1194,280.15253768844224,534
cart,1183,274.6021217244295,509
shipping,1132,273.2893286219082,485
